### Mounting drive on Colab

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


###Libraries

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 13.9 MB/s 


In [3]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Effect of changing learning rate

For studying the effects of learning rate it is varied in the range of 10^-5 to 10^2 

### For ImageNet as a source

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

pre_trained_model = ResNet50(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)
#for layer in pre_trained_model.layers:
#  layer.trainable = False

In [5]:
def compile_model(learning_rate):
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)

  # Add a fully connected layer with "num_neurons" hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)

  x = layers.Dense(1000, activation='relu')(x)

  x = layers.Dense(500, activation='relu')(x)                  
  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=learning_rate), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [6]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [7]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [8]:
for i in [10**-5, 10**-4, 10**-3, 10**-2, 10**-1]:
  print("*********** Fitting model for learning rate of", i ,"***************")
  model = compile_model(i)
  model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
  test_acc = model.evaluate(X_test, y_test)
  print("***********test accuracy is", test_acc,"***************")
  print("***********train time is", sum(cb.logs),"***************")
  print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for learning rate of 1e-05 ***************


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 82s 290ms/step - loss: 2.7821 - acc: 0.0840 - auc: 0.5331 - recall: 0.0024 - precision: 0.1733 - hamming_loss: 0.9976 - f1_score: 0.0046 - val_loss: 2.7219 - val_acc: 0.0677 - val_auc: 0.5081 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 57s 287ms/step - loss: 2.6811 - acc: 0.1061 - auc: 0.5777 - recall: 0.0025 - precision: 0.3675 - hamming_loss: 0.9975 - f1_score: 0.0050 - val_loss: 2.6809 - val_acc: 0.1030 - val_auc: 0.5702 - val_recall: 3.3333e-04 - val_precision: 1.0000 - val_hamming_loss: 0.9997 - val_f1_score: 6.6335e-04
Epoch 3/10
200/200 [==============================] - 58s 290ms/step - loss: 2.6128 - acc: 0.1332 - auc: 0.6231 - recall: 0.0065 - precision: 0.5596 - hamming_loss: 0.9935 - f1_score: 0.0125 - val_loss: 2.6378 - val_acc: 0.1147 - val_auc: 0.6101 - val_recall: 0.0020 - val_precision: 0.5455 - val_h